# 03 — Быстрая генерация изображений и бенчмарки

Цель ноутбука — исследовать скорость и качество генерации на разных настройках и железе:
- стандартный Stable Diffusion vs Turbo/LCM варианты (меньше шагов);
- влияние разрешения, числа шагов и типа сэмплера на время и качество;
- базовый эксперимент с экспортом в ONNX Runtime.

Результат ноутбука:
- таблица/графики со временем генерации и субъективной оценкой качества;
- рекомендации по настройкам для ноутбука и сервера.


In [ ]:
# Импорты для бенчмарков скорости

import time
from dataclasses import dataclass

import torch
from diffusers import StableDiffusionPipeline


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BASE_MODEL_ID = "runwayml/stable-diffusion-v1-5"


@dataclass
class BenchmarkConfig:
    model_id: str
    steps: int
    height: int
    width: int
    batch_size: int = 1


def load_pipe(model_id: str):
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    )
    pipe = pipe.to(DEVICE)
    pipe.safety_checker = None
    return pipe


def benchmark_generation(pipe, config: BenchmarkConfig, prompt: str = "a scenic landscape"):
    """Простой замер времени одной генерации."""
    start = time.time()
    _ = pipe(
        prompt=[prompt] * config.batch_size,
        num_inference_steps=config.steps,
        height=config.height,
        width=config.width,
    ).images
    end = time.time()
    return end - start

